In [6]:
import numpy as np

In [7]:
list_SD36 = ['6x2pt_LSSTxSO_Y1_1sample','6x2pt_LSSTxSO_Y6_1sample'] # input datav prefix
list_COV36 = ['cov_LSSTxSO_Y1_1sample_6x2pt.npy','cov_LSSTxSO_Y6_1sample_6x2pt.npy']

list_outfiles = ['6x2pt_LSSTxSO_Y1_1sample','6x2pt_LSSTxSO_Y6_1sample','3x2pt_LSSTxSO_Y1_1sample','3x2pt_LSSTxSO_Y6_1sample'] # output datav filename

dir_datav = "datav/"
dir_bary = "datav/"
dir_COV = "cov/"
dir_LPC = "baryons/"

In [78]:
class gen_LPCi():
    def __init__(self,id_SD,outfile,id_include=None):
        '''
            id_SD = 0~1, int
        '''
        self.id_SD = id_SD
        self.id_include = id_include
        self.outfile = outfile
        self.list_bary_scenario = ['_eagle','_illustris','_TNG100','_mb2','_owls_AGN','_HzAGN']
        self.datav_dmo = self.load_datav(dir_datav,bary_scenario="_dmo",isCutted=1)
        print(self.id_SD)
        print(self.id_include)
        
        self.load_COV()
        print(self.id_SD)
        print(self.datav_dmo.shape, self.COV_cut.shape)
        
        self.cal_invL()
        self.Ndata = len(self.datav_dmo)
        self.Nscenario = len(self.list_bary_scenario)
        
        self.build_Ratio()
        self.build_Delta()
        self.build_DeltaChy()
        self.SVD(in_Delta=self.DeltaChy)
        self.write_LPC()
        
    
    def load_datav(self,dir_path,bary_scenario,isCutted=0):
        fname_datav = dir_path+list_SD36[self.id_SD]+bary_scenario
        print(fname_datav)
        datav = np.genfromtxt(fname_datav,skip_header=0,dtype="int,double",usecols=[0,1],names=["ind","obs"])["obs"]
        if self.id_include is None:
            self.id_include = np.arange(datav.size)
        datav = datav[self.id_include]
        
        self.Ndata_full  = len(datav)
        self.takeout_ID  = np.where(datav > 1e-20)[0]
        self.zero_ID     = np.where(datav < 1e-20)[0]
        
        if isCutted==1:
            return np.array(datav)[self.takeout_ID]
        else:
            return np.array(datav)
        
            
    def load_COV(self):
        
        self.fname_COV = dir_COV+list_COV36[self.id_SD]
        print(self.fname_COV)
        self.COV_full = np.load(self.fname_COV)
        self.COV_full = self.COV_full[self.id_include][:,self.id_include]
        
        self.COV_cut  = self.COV_full[self.takeout_ID][:,self.takeout_ID]
        
    def output_invCOV(self, outfile, sub_ind=None):
        
        self.invcov_masked = np.zeros((self.Ndata_full,self.Ndata_full))
        invcov_cut = np.linalg.inv(self.COV_cut)
        print(invcov_cut.shape)
        print(self.takeout_ID.size)
        for i in range(self.takeout_ID.size):
            (self.invcov_masked[self.takeout_ID[i]])[self.takeout_ID] = invcov_cut[i]
        if sub_ind is None:
            narr = np.arange(self.Ndata_full)
            col1 = np.repeat(narr, self.Ndata_full)
            col2 = np.tile(narr, self.Ndata_full)
            np.savetxt(outfile, np.c_[col1, col2, self.invcov_masked.flatten()], fmt='%d %d %le')
        else:
            Nsub = sub_ind.size
            narr = np.arange(Nsub)
            col1 = np.repeat(narr, Nsub)
            col2 = np.tile(narr, Nsub)
            np.savetxt(outfile, np.c_[col1, col2, (self.invcov_masked[sub_ind][:,sub_ind]).flatten()], fmt='%d %d %le')
        print('inv cov saved!')
    
    def cal_invL(self):
        self.L    = np.linalg.cholesky(self.COV_cut)
        self.invL =  np.linalg.inv(self.L)
        
    def build_Ratio(self):
        self.Ratio     = np.zeros((self.Ndata,self.Nscenario))
        
        for j in range(self.Nscenario):
        
            datav_bary = self.load_datav(dir_bary,bary_scenario=self.list_bary_scenario[j],isCutted=1)
            self.Ratio.T[j] = datav_bary/self.datav_dmo
            
    def build_Delta(self):
        DeltaT = self.Ratio.T*self.datav_dmo-self.datav_dmo
        self.Delta = DeltaT.T
    
    def build_DeltaChy(self):
        self.DeltaChy = np.dot(self.invL,self.Delta)
        
    def SVD(self,in_Delta):
        # self.U stores PC modes
        # PC1 = self.U.T[0]  ; PC2 = self.U.T[1]
        self.U, self.Sdig, VT = np.linalg.svd(in_Delta,full_matrices=True)
    
    def gen_datav_cosmolike_format(self,datav_cut):
        
        datav_full = np.zeros(self.Ndata_full)
        datav_full[self.takeout_ID] = datav_cut
        
        return datav_full

    
    def write_LPC(self):
        
        self.fname_out = dir_LPC+'LPC_'+self.outfile
        
        self.LPC1 = self.gen_datav_cosmolike_format(np.dot(self.L,self.U.T[0]))
        self.LPC2 = self.gen_datav_cosmolike_format(np.dot(self.L,self.U.T[1]))
        self.LPC3 = self.gen_datav_cosmolike_format(np.dot(self.L,self.U.T[2]))
        self.LPC4 = self.gen_datav_cosmolike_format(np.dot(self.L,self.U.T[3]))
        self.LPC5 = self.gen_datav_cosmolike_format(np.dot(self.L,self.U.T[4]))
        self.LPC6 = self.gen_datav_cosmolike_format(np.dot(self.L,self.U.T[5]))
        
        col_names='LPC1 LPC2 LPC3 LPC4 LPC5 LPC6'
        
        LPC_Matrix=np.column_stack((self.LPC1,self.LPC2,self.LPC3,self.LPC4,self.LPC5,self.LPC6))

        np.savetxt(self.fname_out,LPC_Matrix, fmt='%.8e %.8e %.8e %.8e %.8e %.8e')

       

In [79]:
PCA0 = gen_LPCi(0, list_outfiles[0])
PCA1 = gen_LPCi(1, list_outfiles[1])

datav/6x2pt_LSSTxSO_Y1_1sample_dmo
0
[   0    1    2 ... 1962 1963 1964]
cov/cov_LSSTxSO_Y1_1sample_6x2pt.npy
0
(1509,) (1509, 1509)
datav/6x2pt_LSSTxSO_Y1_1sample_eagle
datav/6x2pt_LSSTxSO_Y1_1sample_illustris
datav/6x2pt_LSSTxSO_Y1_1sample_TNG100
datav/6x2pt_LSSTxSO_Y1_1sample_mb2
datav/6x2pt_LSSTxSO_Y1_1sample_owls_AGN
datav/6x2pt_LSSTxSO_Y1_1sample_HzAGN
datav/6x2pt_LSSTxSO_Y6_1sample_dmo
1
[   0    1    2 ... 1962 1963 1964]
cov/cov_LSSTxSO_Y6_1sample_6x2pt.npy
1
(1526,) (1526, 1526)
datav/6x2pt_LSSTxSO_Y6_1sample_eagle
datav/6x2pt_LSSTxSO_Y6_1sample_illustris
datav/6x2pt_LSSTxSO_Y6_1sample_TNG100
datav/6x2pt_LSSTxSO_Y6_1sample_mb2
datav/6x2pt_LSSTxSO_Y6_1sample_owls_AGN
datav/6x2pt_LSSTxSO_Y6_1sample_HzAGN


In [74]:
PCA0.output_invCOV('invcov_Y1_6x2pt_1sample')

(1509, 1509)
1509
inv cov saved!


In [80]:
PCA0.output_invCOV('invcov_Y1_3x2pt_1sample', np.arange(1650))

(1509, 1509)
1509
inv cov saved!


In [81]:
PCA1.output_invCOV('invcov_Y6_6x2pt_1sample')
PCA1.output_invCOV('invcov_Y6_3x2pt_1sample', np.arange(1650))

(1526, 1526)
1526
inv cov saved!
(1526, 1526)
1526
inv cov saved!


In [12]:
def cal_Qexp(PCA, scenario):
    datav_bary = PCA.load_datav(dir_path=dir_datav, bary_scenario='_'+scenario, isCutted=1)
    diff = datav_bary - PCA.datav_dmo
    diff_chy = np.dot(PCA.invL, diff)
    Qexp = np.dot(PCA.U.T,diff_chy)
    return Qexp

In [15]:
baryons = ['dmo','eagle','illustris','TNG100','mb2','owls_AGN','HzAGN']
Qexps = [None]*7
for i in range(7):
    Qexps[i] = cal_Qexp(PCA0, scenario=baryons[i])
for i in range(7):
    print(baryons[i], (Qexps[i])[0:4])

datav/6x2pt_LSSTxSO_Y1_1sample_dmo
datav/6x2pt_LSSTxSO_Y1_1sample_eagle
datav/6x2pt_LSSTxSO_Y1_1sample_illustris
datav/6x2pt_LSSTxSO_Y1_1sample_TNG100
datav/6x2pt_LSSTxSO_Y1_1sample_mb2
datav/6x2pt_LSSTxSO_Y1_1sample_owls_AGN
datav/6x2pt_LSSTxSO_Y1_1sample_HzAGN
dmo [0. 0. 0. 0.]
eagle [ 3.456611    1.62711608 -0.27714792 -0.40454005]
illustris [32.05850093 -1.89998756 -0.35409335 -0.05223627]
TNG100 [ 4.34292504  1.26050325 -0.42750571 -0.21162027]
mb2 [-0.69363915 -0.90890764  0.73642071 -0.39654304]
owls_AGN [23.45113748  1.59793461  0.62864714  0.11534034]
HzAGN [ 6.49379009  1.80300505 -0.01006508  0.15585429]


In [16]:
Qexps = [None]*7
for i in range(7):
    Qexps[i] = cal_Qexp(PCA1, scenario=baryons[i])
for i in range(7):
    print(baryons[i], (Qexps[i])[0:4])

datav/6x2pt_LSSTxSO_Y6_1sample_dmo
datav/6x2pt_LSSTxSO_Y6_1sample_eagle
datav/6x2pt_LSSTxSO_Y6_1sample_illustris
datav/6x2pt_LSSTxSO_Y6_1sample_TNG100
datav/6x2pt_LSSTxSO_Y6_1sample_mb2
datav/6x2pt_LSSTxSO_Y6_1sample_owls_AGN
datav/6x2pt_LSSTxSO_Y6_1sample_HzAGN
dmo [0. 0. 0. 0.]
eagle [ 6.7738321   3.09365026  0.63754419 -0.68183645]
illustris [57.01858866 -3.46007771  0.8776926  -0.1088936 ]
TNG100 [ 8.11118841  2.36521868  0.96303978 -0.36382013]
mb2 [-1.18221791 -1.563422   -1.6866233  -0.7042721 ]
owls_AGN [43.66981997  2.64192559 -1.47324631  0.21117609]
HzAGN [12.61655572  3.16468505  0.01328706  0.29516724]


In [17]:
nsource = 10
nshear = nsource*(nsource+1)/2
ntheta = 15
nlens = 10

ndata3x2pt= 1965 - (nlens + nsource + 1)*ntheta
PCA3x2pt_0 = gen_LPCi(0, list_outfiles[2], np.arange(ndata3x2pt))

ndata3x2pt= 1965 - (nlens + nsource + 1)*ntheta
PCA3x2pt_1 = gen_LPCi(1, list_outfiles[3], np.arange(ndata3x2pt))

datav/6x2pt_LSSTxSO_Y1_1sample_dmo
0
[   0    1    2 ... 1647 1648 1649]
cov/cov_LSSTxSO_Y1_1sample_6x2pt.npy
0
(1254,) (1254, 1254)
datav/6x2pt_LSSTxSO_Y1_1sample_eagle
datav/6x2pt_LSSTxSO_Y1_1sample_illustris
datav/6x2pt_LSSTxSO_Y1_1sample_TNG100
datav/6x2pt_LSSTxSO_Y1_1sample_mb2
datav/6x2pt_LSSTxSO_Y1_1sample_owls_AGN
datav/6x2pt_LSSTxSO_Y1_1sample_HzAGN
datav/6x2pt_LSSTxSO_Y6_1sample_dmo
1
[   0    1    2 ... 1647 1648 1649]
cov/cov_LSSTxSO_Y6_1sample_6x2pt.npy
1
(1268,) (1268, 1268)
datav/6x2pt_LSSTxSO_Y6_1sample_eagle
datav/6x2pt_LSSTxSO_Y6_1sample_illustris
datav/6x2pt_LSSTxSO_Y6_1sample_TNG100
datav/6x2pt_LSSTxSO_Y6_1sample_mb2
datav/6x2pt_LSSTxSO_Y6_1sample_owls_AGN
datav/6x2pt_LSSTxSO_Y6_1sample_HzAGN


In [18]:
Qexps = [None]*7
for i in range(7):
    Qexps[i] = cal_Qexp(PCA3x2pt_0, scenario=baryons[i])
for i in range(7):
    print(baryons[i], (Qexps[i])[0:4])

datav/6x2pt_LSSTxSO_Y1_1sample_dmo
datav/6x2pt_LSSTxSO_Y1_1sample_eagle
datav/6x2pt_LSSTxSO_Y1_1sample_illustris
datav/6x2pt_LSSTxSO_Y1_1sample_TNG100
datav/6x2pt_LSSTxSO_Y1_1sample_mb2
datav/6x2pt_LSSTxSO_Y1_1sample_owls_AGN
datav/6x2pt_LSSTxSO_Y1_1sample_HzAGN
dmo [0. 0. 0. 0.]
eagle [ 3.44293757  1.61667994 -0.26892889 -0.39729399]
illustris [31.60455975 -1.8730469  -0.35415214 -0.05452694]
TNG100 [ 4.31146524  1.25145149 -0.42205671 -0.20891445]
mb2 [-0.7074309  -0.89304502  0.73832195 -0.38785248]
owls_AGN [23.19967429  1.55917249  0.62600195  0.11743608]
HzAGN [ 6.45844805  1.76993657 -0.00964917  0.15375645]


In [19]:
Qexps = [None]*7
for i in range(7):
    Qexps[i] = cal_Qexp(PCA3x2pt_1, scenario=baryons[i])
for i in range(7):
    print(baryons[i], (Qexps[i])[0:4])

datav/6x2pt_LSSTxSO_Y6_1sample_dmo
datav/6x2pt_LSSTxSO_Y6_1sample_eagle
datav/6x2pt_LSSTxSO_Y6_1sample_illustris
datav/6x2pt_LSSTxSO_Y6_1sample_TNG100
datav/6x2pt_LSSTxSO_Y6_1sample_mb2
datav/6x2pt_LSSTxSO_Y6_1sample_owls_AGN
datav/6x2pt_LSSTxSO_Y6_1sample_HzAGN
dmo [0. 0. 0. 0.]
eagle [ 6.70667775  3.05631542  0.60305419 -0.6585439 ]
illustris [55.36431783 -3.36021326  0.88251907 -0.11504096]
TNG100 [ 7.97993676  2.34085612  0.93713952 -0.35226939]
mb2 [-1.21261808 -1.53298437 -1.68252843 -0.68000078]
owls_AGN [42.68298526  2.50650717 -1.4643728   0.21658973]
HzAGN [1.24477706e+01 3.05389428e+00 6.48051377e-03 2.83393367e-01]
